Source practice: https://www.youtube.com/watch?v=Jy4wM2X21u0&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=3

##Imports

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Neural Network Model

In [3]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [4]:
model = NN(784, 10)

In [5]:
x = torch.rand(64, 784)

In [7]:
print(model(x).shape)

torch.Size([64, 10])


##Set device

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##Hyperparameters

In [12]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

##Load data

In [15]:
train_dataset = datasets.MNIST(download = True, root = 'dataset/', train = True, transform = transforms.ToTensor())

In [16]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

In [17]:
test_dataset = datasets.MNIST(download = True, root = 'dataset/', train = False, transform = transforms.ToTensor())

In [18]:
test_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

##Initialize network

In [19]:
model = NN(input_size = input_size, num_classes = num_classes).to(device)

##Loss and Optimizer

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train Network

In [24]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    #Get data to cuda if possible
    data = data.to(device = device)
    targets = targets.to(device = device)

    #Get the correct shape
    data = data.reshape(data.shape[0], -1) #flatten to a single dimension

    #Forward
    scores = model(data)
    loss = criterion(scores, targets)

    #Backward
    optimizer.zero_grad()
    loss.backward()

    #Gradient descent or adam step
    optimizer.step()


## Check accuracy on training and test

In [29]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1) #maximum values
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()
  return num_correct

In [30]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 56204 / 60000 with accuracy 93.67
Checking accuracy on training data
Got 56204 / 60000 with accuracy 93.67


tensor(56204)